This script is used to test the modules.

In [1]:
%matplotlib inline

import numpy as np
import sys
sys.path.append("../")
import scipy
from netCDF4 import Dataset
import matplotlib
import matplotlib.pyplot as plt
import os
import platform
#import statistics
import meta.ORAS4

print (platform.architecture())
print (os.path)

('64bit', '')
<module 'posixpath' from '/home/yang/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/posixpath.py'>


In [2]:
if __name__=="__main__":
    # sample
    ################################   Input zone  ######################################
    # specify data path
    datapath_ORAS4 = '/mnt/Associate/DataBase/ORAS/ORAS4/Monthly/model'
    #datapath_ERAI = '/project/Reanalysis/ERA_Interim/Subdaily/Model'
    output_path = '/home/yang/workbench/test'
    #output_path = '/project/Reanalysis/ERA_Interim/Subdaily/Model'
    mask_path = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ORAS4/mesh_mask.nc'
    mask_subbasin_path = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/land_sea_mask/mask_subbasin_ORCA_MOM.nc'
    #####################################################################################
    print ('*********************** call functions *************************')
    instance = meta.ORAS4.oras(datapath_ORAS4,output_path)
    instance.ohc(1984,1984,mask_path, mask_subbasin_path)


*********************** call functions *************************
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
Start quantifying the meridional energy transport / ocean heat content in the ocean.
